In [ ]:
import numpy as np

import pandas as pd 

from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
# Read the taining data
sales_train = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv",parse_dates=["date"],dayfirst=True)
items = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")
sales_test = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")
shops = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")

print(f'sales_train columns are{sales_train.columns}',
      f'items columns are {items.columns}',
      f'sales_test columns are {sales_test.columns}',
      f'shops columns are {shops.columns}',
      sep='\n')


In [ ]:
# merging the categories in order to increase probability of finding patterns
# Shop CSV data is negligible

df= sales_train[["item_id","item_price"]]
df2= df.drop_duplicates(subset="item_id",keep="first")
df3= df2.sort_values("item_id")
df3.reset_index(inplace=True,drop=True)
items_updated = pd.merge(items,df3,on="item_id",how="left")
items_updated["item_price"]=items_updated["item_price"].replace(np.nan, 0)

sales_train_1= pd.merge(sales_train,
                       items,
                       on="item_id",
                       how="inner")
sales_train_1["month_id"] = pd.DatetimeIndex(sales_train_1["date"]).month

sales_train_1["year_id"] = pd.DatetimeIndex(sales_train_1["date"]).year

# sales_train_1.head()

sales_train_1.info()

In [ ]:
features =["month_id","shop_id","item_id","item_category_id"]

X = sales_train_1[features]
y= sales_train_1["item_cnt_day"]

train_X, val_X, train_y, val_y = train_test_split(X,y, random_state=0)

# checking the conditions with decision tree regressor to implement in "RandomForestRegressor"

# criterion taken from sklearn Documentation
criterion =["mse", "friedman_mse", "mae", "poisson"]
#Best is mse. Mae just takes forever to train(donot use "mae")** "poisson not tested"
    
def score_mae(train_X,train_y,val_X,val_y):
    model_train= DecisionTreeRegressor(criterion="mse",random_state=0,max_leaf_nodes=55100,min_samples_split=12)
    model_train.fit(train_X,train_y)
    predict_train=model_train.predict(val_X)
    print("Max_depth is ",model_train.tree_.max_depth) #Using the attribute"model.tree_" 
    return mean_absolute_error(predict_train,val_y)


# for i in range(100,100000,5000): #for checking best "max_leaf_nodes" is 55100
#     score=score_mae(i,train_X,train_y,val_X,val_y)
#     print(f"Mae for node {i} is {score}")
    
# for i in range(2,50,1): # for checking best "min_samples_split" is 12
#     score=score_mae(i,train_X,train_y,val_X,val_y)
#     print(f"Mae for split {i} is {score}")

# for i in criterion[:2]:
#     score=score_mae(i,train_X,train_y,val_X,val_y)
#     print(f"Mae for nodes {i} is {score}")


In [ ]:
model_train_rfr = RandomForestRegressor(n_estimators=100,criterion="mse",random_state=0,verbose=2,max_leaf_nodes=55100,min_samples_split=12)
model_train_rfr.fit(train_X,train_y)
predict_train_rfr = model_train_rfr.predict(val_X)


In [ ]:
mean_absolute_error(predict_train_rfr,val_y)


In [ ]:

sales_test["month_id"]= 11

test_X_1= pd.merge(sales_test,items_updated,on="item_id",how="left")
test_X= test_X_1[features]

prediction_final = model_train_rfr.predict(test_X)



In [ ]:
output = pd.DataFrame({"ID": sales_test["ID"],
                       "item_cnt_month" : prediction_final})

output.to_csv("submission.csv",index=False)
output